In [ ]:
import os
from tqdm import tqdm
from PIL import Image

In [ ]:
target_size = 512
img_path_origin = '/media/trinhnh1/3A08638408633DCF/kaggle/human-protein/input/HPA_v18_RGBY/img/train/'
img_path_resize = '/media/trinhnh1/3A08638408633DCF/kaggle/human-protein/input/HPA_v18_RGBY/img/train_resize/'
img_lists = os.listdir(img_path_origin)

In [ ]:
# version 1: one thread
# convert image to 512*512 then save it
for img in tqdm(img_lists):
    img_origin = Image.open(img_path_origin + img)
    img_resize = img_origin.resize((target_size, target_size), Image.CUBIC)
    img_resize.save(img_path_resize+img)

In [ ]:
# version 2: multiple threads
from PIL import Image, ImageOps
from PIL import ImageFile
import glob
from shutil import copyfile
from multiprocessing import Process
from multiprocessing import Pool

ImageFile.LOAD_TRUNCATED_IMAGES = True

def transfer(image_path):
    output_path = '/media/trinhnh1/3A08638408633DCF/kaggle/human-protein/input/test_img/
    if str(image_path.split('/')[-1][:-4] + '.jpg') in glob.glob(output_path + '.jpg'):
        return
    im = Image.open(image_path)
    im.save(output_path + image_path.split('/')[-1][:-4] + '.jpg')

def full_pading(image_path, desired_size=512):
    output_path = '/media/trinhnh1/3A08638408633DCF/kaggle/human-protein/input/HPA_v18_RGBY/img/train_resize/'
    im = Image.open(image_path)
    old_size = im.size  # old_size[0] is in (width, height) format

    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])

    # using thumbnai() or resize() method to resize the input image and keep its aspect ratio
    im.thumbnail(new_size, Image.ANTIALIAS)

    # or we can expand the resized image by adding borders to its 4 side
    delta_w = desired_size - new_size[0]
    delta_h = desired_size - new_size[1]
    padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))

    new_im = ImageOps.expand(im, padding, fill="black")
    new_im.save(output_path + image_path.split('/')[-1])

def preprocessing():
    input_path = '/media/trinhnh1/3A08638408633DCF/kaggle/human-protein/input/HPA_v18_RGBY/img/train/'
    with Pool(40) as p:
        p.map(full_pading, glob.glob(input_path + '/*.jpg'))
    '''
    procs = []
    for image_path in glob.glob(input_path + '/*'):
        proc = Process(target=full_pading, args=(image_path,))
        procs.append(proc)
        proc.start()
        proc.join()
    '''
def png_to_jpg():
    raw_input_path = '/media/trinhnh1/3A08638408633DCF/kaggle/human-protein/input/test/
    with Pool(40) as p:
        p.map(transfer, glob.glob(raw_input_path + '/*.png'))

if __name__ == '__main__':
    preprocessing()
    
    #png_to_jpg()